In [1]:
import pandas as pd

In [2]:
# 导入文件
df_table = pd.read_csv("2_compare_GBK_11.29.csv", skiprows = 0, encoding="gbk")
df_table.head()

,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,c_admin_type,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y
0,65,400223,佑國軍節度使,904,907,關內道,佑國軍節度,618,907,關內道,Jiedu,general,True,False,False,False,4,290,286
1,75,14548,雍州,618,690,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,73,144,71
2,85,14548,雍州,690,713,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,24,144,120
3,90,14490,京兆府,713,907,關內道,京兆府,713,907,關內道,Fu,common,True,True,False,False,195,195,0
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,Xian,common,False,True,False,False,290,45,245


### 筛选出multiple对应和非multiple的

In [3]:
# ID筛选：先在旧dataframe里面添加一个new_id计数的列，然后只保存count为1的部分
ID_list=df_table['new_id'].tolist() #把new_id保存成一个list，便于计数
df_table['new_id_count']=None
# 获得new_id出现的次数，并计入dataframe中
for i in range(len(df_table)):
    df_table['new_id_count'][i]=ID_list.count(df_table['new_id'][i])
df_table.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,c_admin_type,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count
0,65,400223,佑國軍節度使,904,907,關內道,佑國軍節度,618,907,關內道,Jiedu,general,True,False,False,False,4,290,286,1
1,75,14548,雍州,618,690,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,73,144,71,1
2,85,14548,雍州,690,713,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,24,144,120,1
3,90,14490,京兆府,713,907,關內道,京兆府,713,907,關內道,Fu,common,True,True,False,False,195,195,0,1
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,Xian,common,False,True,False,False,290,45,245,3


In [5]:
# 创建一个空的dataframe来保存进行所有对比之前就没有multiple的数据
output_once_original = df_table[df_table['new_id_count']==1]
df_table_m=df_table[df_table['new_id_count']!=1] # 创建新的df_table_m,只保留multiple的

print("原数据计数： " + str(len(df_table)))
print("multiple计数： " + str(len(df_table_m)))
print("非multiple计数： " + str(len(output_once_original)))

output_once_original.head()

原数据计数： 4730
multiple计数： 1661
非multiple计数： 3069


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,c_admin_type,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count
0,65,400223,佑國軍節度使,904,907,關內道,佑國軍節度,618,907,關內道,Jiedu,general,True,False,False,False,4,290,286,1
1,75,14548,雍州,618,690,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,73,144,71,1
2,85,14548,雍州,690,713,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,24,144,120,1
3,90,14490,京兆府,713,907,關內道,京兆府,713,907,關內道,Fu,common,True,True,False,False,195,195,0,1
7,100,14899,乾封,666,703,關內道,乾封,705,907,河南道,Xian,common,False,False,False,True,38,203,165,1


### 一、层级对比
* 对前一步multiple的数据进行对比，不符合的删除，最后得到的数据储存在df_table_m_1，被删掉的数据储存在df_table_1_deleted_cengji

In [7]:
# admin type全部小写
df_table_m_1=df_table_m # df_table_m_1 保存进行层级对比筛选后留下的数据
m1_index_list=df_table_m_1.index.tolist() # 提取index的内容到一个list里面

# 保险起见，把行政区的type全都变成小写
# 进行删除操作的multiple数据
for i in m1_index_list:
    df_table_m_1['c_admin_type'][i]=df_table_m_1['c_admin_type'][i].lower() 
# 最初就只有单个new_id的数据
for i in output_once_original.index.tolist():
    output_once_original['c_admin_type'][i]=output_once_original['c_admin_type'][i].lower() 

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self.user_ns)
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by InteractiveShellApp.init_path()


In [8]:
df_table_m_1_deleted=df_table_m_1 #再创建一个dataframe来保存层级对比之后被删除的数据

In [9]:
# 对multiple的记录进行step① 层级的对比【删除不一致的】
fuzhou_list=['fu','jun (zhou)', 'jun commandery', 'zhou','zhou (jun)']
for i in df_table_m_1.index.tolist(): 
    a=df_table_m_1['guo_name_standard_by_yun'][i]
    b=df_table_m_1['c_admin_type'][i]
    if '都督府'in a:
        if b!='dudufu':
            df_table_m_1=df_table_m_1.drop([i])
            continue
        else:
            continue
    elif '總管府' in a:
        if b!='zongguanfu':
            df_table_m_1=df_table_m_1.drop([i])
            continue
        else:
            continue
    elif '節度使' in a:
        if b!='jiedu':
            df_table_m_1=df_table_m_1.drop([i])
            continue
        else:
            continue
    elif '都護府' in a:
        if b!='duhufu':
            df_table_m_1=df_table_m_1.drop([i])
            continue
        else:
            continue
    elif '道' in a:
        if a!=df_table_m_1['c_name'][i]:
            df_table_m_1=df_table_m_1.drop([i])
            continue
        else:
            continue
    elif ((a[-1]=='府') or (a[-1]=='州') or (a[-1]=='郡')):
        if b not in fuzhou_list:
            df_table_m_1=df_table_m_1.drop([i])
            continue
        else:
            continue
    elif b != 'xian':
        df_table_m_1=df_table_m_1.drop([i])
                                
print("层级对比后剩下的数据条目数： " + str(len(df_table_m_1)))
df_table_m_1.head() 
#这里df_table_m_1保存的是层级对比之后剩下的数据

层级对比后剩下的数据条目数： 1280


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,c_admin_type,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,xian,common,False,True,False,False,290,45,245,3
5,95,14494,長安,618,907,關內道,長安,748,907,關內道,xian,common,False,True,False,False,290,160,130,3
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,xian,common,False,True,False,False,290,205,85,3
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,xian,common,False,True,False,False,11,10,1,2
10,110,14611,咸寧,748,758,關內道,咸寧,618,907,關內道,xian,general,True,False,False,False,11,290,279,2


In [11]:
df_table_m_1_deleted_cengji=df_table_m_1_deleted.drop(df_table_m_1.index)
print("层级对比后被删除的数据条目数： " + str(len(df_table_m_1_deleted_cengji)))
df_table_m_1_deleted_cengji.head()

层级对比后被删除的数据条目数： 381


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,c_admin_type,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count
35,210,400441,始平,618,709,關內道,始平郡,742,757,唐朝,jun commandery,common,False,False,False,True,92,16,76,2
39,225,400376,扶風,620,627,關內道,扶風郡,742,755,關內道,jun commandery,common,False,False,False,True,8,14,6,2
65,420,14528,馮翊郡,742,756,關內道,馮翊,618,907,關內道,xian,general,True,False,False,False,15,290,275,2
68,430,14528,馮翊郡,757,758,關內道,馮翊,618,907,關內道,xian,general,True,False,False,False,2,290,288,2
72,440,400369,馮翊,618,907,關內道,馮翊郡,742,757,關內道,jun commandery,common,False,True,False,False,290,16,274,2


In [85]:
df_table_m_1_deleted_cengji.to_csv('cengji_deleted.csv', index=False, encoding='GBK')

* 对最开始就no-multiple的数据**output_once_original**进行层级标记，层级一致的标记TRUE，不一致的FALSE，层级一致性的列，命名为：admin_same
* 这里其实可以把下面这个cell的code定义成一个函数，这样前一步也可以调用这个函数赋值True/False，之后只需要把False的内容删除即可

In [16]:
# 对non-multiple的记录进行① 层级的对比，标记是否一致
fuzhou_list=['fu','jun (zhou)', 'jun commandery', 'zhou','zhou (jun)']
output_once_original['admin_same']=None 
for i in output_once_original.index.tolist(): 
    a=output_once_original['guo_name_standard_by_yun'][i]
    b=output_once_original['c_admin_type'][i]
    if '都督府'in a:
        output_once_original['admin_same'][i]=(b=='dudufu')
        continue
    elif '總管府' in a:
        output_once_original['admin_same'][i]=(b=='zongguanfu')
        continue
    elif '節度使' in a:
        output_once_original['admin_same'][i]=(b=='jiedu')
        continue
    elif '都護府' in a:
        output_once_original['admin_same'][i]=(b=='duhufu')
        continue
    elif '道' in a:
        output_once_original['admin_same'][i]=(a==output_once_original['c_name'][i])
        continue
    elif ((a[-1]=='府') or (a[-1]=='州') or (a[-1]=='郡')):
        output_once_original['admin_same'][i]=(b in fuzhou_list)
        continue
    else:
        output_once_original['admin_same'][i]=(b=='xian')
                                
output_once_original.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global

,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count,admin_same
0,65,400223,佑國軍節度使,904,907,關內道,佑國軍節度,618,907,關內道,...,general,True,False,False,False,4,290,286,1,True
1,75,14548,雍州,618,690,關內道,雍州,618,761,關內道,...,common,True,False,False,False,73,144,71,1,True
2,85,14548,雍州,690,713,關內道,雍州,618,761,關內道,...,common,True,False,False,False,24,144,120,1,True
3,90,14490,京兆府,713,907,關內道,京兆府,713,907,關內道,...,common,True,True,False,False,195,195,0,1,True
7,100,14899,乾封,666,703,關內道,乾封,705,907,河南道,...,common,False,False,False,True,38,203,165,1,True


In [21]:
output_once_original.drop(columns=['new_id_count']).to_csv('original_once_cengji_gbk.csv', encoding='gbk')
# 这个文件是只保存初始非multipel进行层级判断之后的结果【其实可以不保存到文件，因为后面只加不减，所以如果不想要某些数据，csv里面直接删就行】

#### 把层级对比后非multiple的数据存入output_once_1，进行后面的对比分析标记（+2 columns）
#### 层级对比后multiple的数据存入df_table_m_2，进行后面的对比标记【要考虑multiple id的问题】（+6 columns）

In [32]:
# 获得new_id出现的次数，并计入dataframe中
ID_list_1=df_table_m_1['new_id'].tolist() #把new_id保存成一个list，便于计数
df_table_m_1['new_id_count_1']=None
for i in df_table_m_1.index.tolist():
    df_table_m_1['new_id_count_1'][i]= ID_list_1.count(df_table_m_1['new_id'][i])

# 创建一个空的dataframe来保存pick出来的没有multiple的数据
output_once_1 = df_table_m_1[df_table_m_1['new_id_count_1']==1]
df_table_m_2=df_table_m_1[df_table_m_1['new_id_count_1']!=1] # 只保留multiple的作為2，放到下一步
print("层级对比后剩下的multiple数据条目数： " + str(len(df_table_m_2)))
print(len(output_once_1))
output_once_1.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


层级对比后剩下的multiple数据条目数： 1063
217


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count,new_id_count_1
34,210,400343,始平,618,709,關內道,始平,618,708,關內道,...,common,False,True,False,False,92,91,1,2,1
38,225,14540,扶風,620,627,關內道,扶風,634,907,關內道,...,common,False,False,False,True,8,274,266,2,1
66,420,400369,馮翊郡,742,756,關內道,馮翊郡,742,757,關內道,...,common,True,False,False,False,15,16,1,2,1
69,430,400369,馮翊郡,757,758,關內道,馮翊郡,742,757,關內道,...,common,False,False,True,False,2,16,14,2,1
71,440,14528,馮翊,618,907,關內道,馮翊,618,907,關內道,...,general,True,True,False,False,290,290,0,2,1


### 二、名称是否一致

In [35]:
df_table_m_3=df_table_m_2 # df_table_m_3保存name对比对应的2个columns

In [36]:
# 定义一个名称比较的函数
def name_compare(df):
    df['name_same']=None
    for i in df.index.tolist():
        a=df['guo_name_standard_by_yun'][i]
        b=df['c_name'][i]
        df['name_same'][i]=(a==b)
    return df

df_table_m_3=name_compare(df_table_m_3)
output_once_1_name=name_compare(output_once_1)
output_once_original_name=name_compare(output_once_original)

print(len(df_table_m_3))
df_table_m_3.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path whil

1063


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count,new_id_count_1,name_same
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,...,False,True,False,False,290,45,245,3,3,True
5,95,14494,長安,618,907,關內道,長安,748,907,關內道,...,False,True,False,False,290,160,130,3,3,True
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,False,True,False,False,290,205,85,3,3,True
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,False,True,False,False,11,10,1,2,2,True
10,110,14611,咸寧,748,758,關內道,咸寧,618,907,關內道,...,True,False,False,False,11,290,279,2,2,True


In [49]:
# length check
print(len(output_once_1_name))
print(len(output_once_original_name))

217
3069


In [55]:
pd.value_counts(df_table_m_3['name_same']) # name是否一致的统计结果

True     966
False     97
Name: name_same, dtype: int64

*  'name_same'根据值分为两类：  
1）对于‘name-same’为False的，【保存为子df1】'new_id_count_name'=0  
2）对于‘name_same’为True的，【保存为子df2】统计这里面的new_id  
* 最后再合并df1和df2

In [58]:
df_table_m_3['new_id_count_name']=None
df_table_m_3_false=df_table_m_3[df_table_m_3['name_same']==False]
df_table_m_3_false['new_id_count_name']='0'

# 进行计数分类
df_table_m_3_true=df_table_m_3[df_table_m_3['name_same']==True]
ID_list_3=df_table_m_3_true['new_id'].tolist() #把new_id保存成一个list，便于计数
for i in df_table_m_3_true.index.tolist():
    df_table_m_3_true['new_id_count_name'][i]= (ID_list_3.count(df_table_m_3_true['new_id'][i])==1)

pd.value_counts(df_table_m_3_true['new_id_count_name'])

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pan

False    962
True       4
Name: new_id_count_name, dtype: int64

In [59]:
df_table_m_4=df_table_m_3_true.append([df_table_m_3_false])

In [60]:
pd.value_counts(df_table_m_4['new_id_count_name'])

False    962
0         97
True       4
Name: new_id_count_name, dtype: int64

In [61]:
df_table_m_4.head()

,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,new_id_count,new_id_count_1,name_same,new_id_count_name
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,...,True,False,False,290,45,245,3,3,True,False
5,95,14494,長安,618,907,關內道,長安,748,907,關內道,...,True,False,False,290,160,130,3,3,True,False
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,True,False,False,290,205,85,3,3,True,False
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,True,False,False,11,10,1,2,2,True,False
10,110,14611,咸寧,748,758,關內道,咸寧,618,907,關內道,...,False,False,False,11,290,279,2,2,True,False


### 三、所属关系：只pick “道” 一致的

In [62]:
# 定义一个名称比较的函数
def dao_compare(df):
    df['dao_same']=None
    for i in df.index.tolist():
        a=df['guo_belong'][i]
        b=df['c_belong'][i]
        df['dao_same'][i]=(a==b)
    return df

df_table_m_5=dao_compare(df_table_m_4) # df_table_m_5保存“道”对比的结果
output_once_1_dao=dao_compare(output_once_1_name)
output_once_original_dao=dao_compare(output_once_original_name)

print(len(df_table_m_5))
print(len(output_once_1_dao))
print(len(output_once_original_dao))
df_table_m_5.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # This is added back by Interactive

1063
217
3069


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,overlap,empty,guo_y,cbdb_y,different_y,new_id_count,new_id_count_1,name_same,new_id_count_name,dao_same
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,...,False,False,290,45,245,3,3,True,False,True
5,95,14494,長安,618,907,關內道,長安,748,907,關內道,...,False,False,290,160,130,3,3,True,False,True
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,False,False,290,205,85,3,3,True,False,True
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,False,False,11,10,1,2,2,True,False,True
10,110,14611,咸寧,748,758,關內道,咸寧,618,907,關內道,...,False,False,11,290,279,2,2,True,False,True


In [63]:
pd.value_counts(df_table_m_5['dao_same'])

True     628
False    435
Name: dao_same, dtype: int64

In [65]:
df_table_m_5['new_id_count_dao']=None
df_table_m_5_false=df_table_m_5[df_table_m_5['dao_same']==False]
df_table_m_5_false['new_id_count_dao']='0'

# 进行计数分类
df_table_m_5_true=df_table_m_5[df_table_m_5['dao_same']==True]
ID_list_5=df_table_m_5_true['new_id'].tolist() #把new_id保存成一个list，便于计数
for i in df_table_m_5_true.index.tolist():
    df_table_m_5_true['new_id_count_dao'][i]= (ID_list_5.count(df_table_m_5_true['new_id'][i])==1)

df_table_m_6=df_table_m_5_true.append([df_table_m_5_false])
pd.value_counts(df_table_m_6['new_id_count_dao'])

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(c

False    471
0        435
True     157
Name: new_id_count_dao, dtype: int64

In [66]:
df_table_m_6.head()

,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,empty,guo_y,cbdb_y,different_y,new_id_count,new_id_count_1,name_same,new_id_count_name,dao_same,new_id_count_dao
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,...,False,290,45,245,3,3,True,False,True,False
5,95,14494,長安,618,907,關內道,長安,748,907,關內道,...,False,290,160,130,3,3,True,False,True,False
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,False,290,205,85,3,3,True,False,True,False
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,False,11,10,1,2,2,True,False,True,False
10,110,14611,咸寧,748,758,關內道,咸寧,618,907,關內道,...,False,11,290,279,2,2,True,False,True,False


In [69]:
output_once_original_dao.head()

,Unnamed: 0,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,...,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y,admin_same,name_same,dao_same
0,0,65,400223,佑國軍節度使,904,907,關內道,佑國軍節度,618,907,...,True,False,False,False,4,290,286,True,False,True
1,1,75,14548,雍州,618,690,關內道,雍州,618,761,...,True,False,False,False,73,144,71,True,True,True
2,2,85,14548,雍州,690,713,關內道,雍州,618,761,...,True,False,False,False,24,144,120,True,True,True
3,3,90,14490,京兆府,713,907,關內道,京兆府,713,907,...,True,True,False,False,195,195,0,True,True,True
4,7,100,14899,乾封,666,703,關內道,乾封,705,907,...,False,False,False,True,38,203,165,True,True,False


In [81]:
# 非multiple的内容保存到文件
output_once_1_dao.drop(columns=['new_id_count','new_id_count_1']).to_csv('cengji_once_final_gbk.csv', index=False, encoding='gbk')
output_once_original_dao.to_csv('original_once_final_gbk.csv', index=False, encoding='gbk')

### 四、年代对比
起始年相减取绝对值a，终止年相减取绝对值b，最后pick up  min（a+b）

In [71]:
df_table_m_6['year_diff']=None
for i in df_table_m_6.index.tolist(): # 因为这里index不是从0,1,2……这样开始的，所以要先提出index的list来
    a=abs(df_table_m_6['guo_time_fy'][i]-df_table_m_6['c_firstyear_new'][i])
    b=abs(df_table_m_6['guo_time_ly'][i]-df_table_m_6['c_lastyear_new'][i])
    df_table_m_6['year_diff'][i]= a+b # 把年代差的绝对值和存入

df_table_m_6.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,guo_y,cbdb_y,different_y,new_id_count,new_id_count_1,name_same,new_id_count_name,dao_same,new_id_count_dao,year_diff
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,...,290,45,245,3,3,True,False,True,False,245
5,95,14494,長安,618,907,關內道,長安,748,907,關內道,...,290,160,130,3,3,True,False,True,False,130
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,290,205,85,3,3,True,False,True,False,85
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,11,10,1,2,2,True,False,True,False,1
10,110,14611,咸寧,748,758,關內道,咸寧,618,907,關內道,...,11,290,279,2,2,True,False,True,False,279


思路：
* 提取一个new_id的list，里面只保存identical的new_id
* 用这个identical的new_id list来做遍历循环，取dataframe里面对应year_diff最小的值min
* 用new_id list里面正在遍历的new_id和筛出来的最小值做条件，drop掉其他的值

In [73]:
m6_newid_list=df_table_m_6['new_id'].tolist()
m6_newid_list_identical=list(set(m6_newid_list)) # 提取m6里面不重复的new_id
min_value={}
for i in m6_newid_list_identical: # 遍历所有的identical的new_id
    year_diff_min=9999
    for j in df_table_m_6.index.tolist() : # 遍历所有的数据（index）
        if df_table_m_6['new_id'][j]==i: # 判断每个new_id是否是正在循环的new_id
            if year_diff_min > df_table_m_6['year_diff'][j]: 
                #判断对应的年代差，选取最小的赋值给min那个辞典，从而最终获得每个new_id对应的年代差最小值
                 year_diff_min = df_table_m_6['year_diff'][j]
    min_value[i]=year_diff_min # 建立一个字典，保存所有identical的new_id对应的最小年代差
min_value

{6150: 10,
 14345: 1,
 26635: 123,
 24590: 16,
 6160: 2,
 6170: 0,
 30750: 343,
 6175: 27,
 30751: 165,
 30752: 110,
 14375: 6,
 6185: 27,
 12330: 5,
 18475: 102,
 12335: 56,
 16445: 178,
 28735: 25,
 6210: 1,
 6211: 1,
 28740: 71,
 24645: 16,
 10310: 25,
 4165: 401,
 6230: 1,
 20570: 159,
 95: 85,
 26720: 16,
 6240: 22,
 10340: 9,
 10345: 3,
 110: 1,
 16500: 1,
 14460: 232,
 14461: 0,
 6270: 0,
 6275: 147,
 140: 138,
 6285: 147,
 26765: 104,
 150: 1,
 26775: 36,
 24730: 169,
 6300: 0,
 18595: 7,
 18600: 4,
 180: 206,
 190: 10,
 2240: 221,
 200: 1,
 2250: 145,
 14560: 234,
 14561: 0,
 22760: 324,
 14585: 0,
 4345: 285,
 16645: 268,
 2310: 288,
 6405: 260,
 20750: 67,
 16655: 171,
 24850: 4,
 24851: 0,
 20760: 89,
 20775: 116,
 12590: 64,
 12605: 9,
 26950: 287,
 4425: 162,
 29060: 142,
 27015: 120,
 14735: 283,
 14745: 285,
 8605: 0,
 14755: 288,
 2475: 54,
 14765: 288,
 2485: 130,
 2486: 21,
 18870: 16,
 14775: 284,
 2490: 10,
 2491: 21,
 8635: 277,
 2495: 11,
 14785: 259,
 2500: 0,
 

In [79]:
df_table_m_6['year_diff_min']=None
for i in df_table_m_6.index.tolist():
    key_num=df_table_m_6['new_id'][i]
    df_table_m_6['year_diff_min'][i]= (df_table_m_6['year_diff'][i]==min_value[key_num]) # new_id相同，年代差相同就True，不同False

df_table_m_6.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,cbdb_y,different_y,new_id_count,new_id_count_1,name_same,new_id_count_name,dao_same,new_id_count_dao,year_diff,year_diff_min
4,95,14493,長安,618,907,關內道,長安,703,747,關內道,...,45,245,3,3,True,False,True,False,245,False
5,95,14494,長安,618,907,關內道,長安,748,907,關內道,...,160,130,3,3,True,False,True,False,130,False
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,205,85,3,3,True,False,True,False,85,True
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,10,1,2,2,True,False,True,False,1,True
10,110,14611,咸寧,748,758,關內道,咸寧,618,907,關內道,...,290,279,2,2,True,False,True,False,279,False


In [80]:
df_table_m_6['new_id_count_year']=None
df_table_m_6_false=df_table_m_6[df_table_m_6['year_diff_min']==False]
df_table_m_6_false['new_id_count_year']='0'

# 进行计数分类
df_table_m_6_true=df_table_m_6[df_table_m_6['year_diff_min']==True]
ID_list_6=df_table_m_6_true['new_id'].tolist() #把new_id保存成一个list，便于计数
for i in df_table_m_6_true.index.tolist():
    df_table_m_6_true['new_id_count_year'][i]= (ID_list_6.count(df_table_m_6_true['new_id'][i])==1)

df_table_m_final=df_table_m_6_true.append([df_table_m_6_false])
pd.value_counts(df_table_m_final['new_id_count_year'])

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':
F:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2963: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(c

0        554
True     339
False    170
Name: new_id_count_year, dtype: int64

In [82]:
df_table_m_final.head()

,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,different_y,new_id_count,new_id_count_1,name_same,new_id_count_name,dao_same,new_id_count_dao,year_diff,year_diff_min,new_id_count_year
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,85,3,3,True,False,True,False,85,True,True
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,1,2,2,True,False,True,False,1,True,True
15,140,14495,新豐,618,691,關內道,新豐,705,742,關內道,...,36,4,4,True,False,True,False,138,True,True
22,150,400002,新豐,705,748,關內道,新豐,705,747,關內道,...,1,4,4,True,False,True,False,1,True,True
28,190,14515,盩厔,618,742,關內道,盩厔,627,741,關內道,...,10,2,2,True,False,True,False,10,True,True


In [83]:
len(df_table_m_final)

1063

In [84]:
df_table_m_final.drop(columns=['new_id_count','new_id_count_1','year_diff']).to_csv("final_multi_gbk.csv",index=False,encoding='GBK')

#### 下面是以前有删除时候的内容

In [16]:
# 进行计数分类
ID_list_4=df_table_m_4['new_id'].tolist() #把new_id保存成一个list，便于计数
df_table_m_4['new_id_count_4']=None
for i in df_table_m_4.index.tolist():
    df_table_m_4['new_id_count_4'][i]= ID_list_4.count(df_table_m_4['new_id'][i])

# 创建一个空的dataframe来保存pick出来的没有multiple的数据
output_once_4 = df_table_m_4[df_table_m_4['new_id_count_4']==1]
df_table_m_5=df_table_m_4[df_table_m_4['new_id_count_4']!=1] # 只保留multiple的
print("层级对比后剩下的multiple数据条目数： " + str(len(df_table_m_5)))
print("层级对比后剩下的非multiple数据条目数： " + str(len(output_once_4)))
output_once_4.head()

F:\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


层级对比后剩下的multiple数据条目数： 43
层级对比后剩下的非multiple数据条目数： 143


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,...,empty,guo_y,cbdb_y,different_y,new_id_count,new_id_count_1,new_id_count_2,new_id_count_3,year_diff,new_id_count_4
6,95,400001,長安,618,907,關內道,長安,703,907,關內道,...,False,290,205,85,3,3,3,3,85,1
9,110,14491,咸寧,748,758,關內道,咸寧,748,757,關內道,...,False,11,10,1,2,2,2,2,1,1
15,140,14495,新豐,618,691,關內道,新豐,705,742,關內道,...,True,74,38,36,4,4,4,3,138,1
22,150,400002,新豐,705,748,關內道,新豐,705,747,關內道,...,False,44,43,1,4,4,4,3,1,1
28,190,14515,盩厔,618,742,關內道,盩厔,627,741,關內道,...,False,125,115,10,2,2,2,2,10,1


In [17]:
# 清理数据，删除new_id计数栏
output_final_1=output_once_1.drop(columns=['new_id_count_1'])
output_final_2=output_once_2.drop(columns=['new_id_count_1','new_id_count_2'])
output_final_3=output_once_3.drop(columns=['new_id_count_1','new_id_count_2','new_id_count_3'])
output_final_4=output_once_4.drop(columns=['new_id_count_1','new_id_count_2','new_id_count_3','new_id_count_4','year_diff'])

In [18]:
output_final=output_once.append([output_final_1,output_final_2,output_final_3,output_final_4])
output_final=output_final.drop(columns=['new_id_count'])
print(len(output_final))
output_final.head()

3570


,new_id,c_id,guo_name_standard_by_yun,guo_time_fy,guo_time_ly,guo_belong,c_name,c_firstyear_new,c_lastyear_new,c_belong,c_admin_type,c_idtype,guo_in_cbdb,cbdb_in_guo,overlap,empty,guo_y,cbdb_y,different_y
0,65,400223,佑國軍節度使,904,907,關內道,佑國軍節度,618,907,關內道,Jiedu,general,True,False,False,False,4,290,286
1,75,14548,雍州,618,690,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,73,144,71
2,85,14548,雍州,690,713,關內道,雍州,618,761,關內道,Zhou,common,True,False,False,False,24,144,120
3,90,14490,京兆府,713,907,關內道,京兆府,713,907,關內道,Fu,common,True,True,False,False,195,195,0
7,100,14899,乾封,666,703,關內道,乾封,705,907,河南道,Xian,common,False,False,False,True,38,203,165


In [19]:
# 清理数据并导出到文件
df_table_m_final=df_table_m_5.drop(columns=['new_id_count','new_id_count_1','new_id_count_2','new_id_count_3','new_id_count_4','year_diff'])
df_table_m_final.to_csv("two_compare_year_multi_gbk.csv",index=False,encoding='GBK')
output_final.to_csv("two_compare_no_multi_gbk.csv",index=False,encoding='GBK')